# Преобразование данныз для "Хоть потоп"

## Общая информация о проекте и план

### Описание проекта

* Целью работы является защита даных клиентов.
* Необходимо разработать алгоритм преобразования данных, чтобы по ним было сложно восстановить персональную информацию.
* Необходимо, чтобы при преобразовании данных качество модели машинного обучения не ухудшилось.

### План

* Загрузка и изучение данных
* Изменится ли качество модели при умножении признаков на обратимую матрице?
* Алгоритм преобразования данных для решения задачи.
* Реализация алгоритма и проверка качества линейной регресии до и после преобразования. 

## Загрузка библиотек и инициализация констант

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

## Загузка и предобработка данных

In [2]:
try:
    data = pd.read_csv('../../datasets/insurance.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/insurance.csv')

In [3]:
data.head(10)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
5,1,43.0,41000.0,2,1
6,1,39.0,39700.0,2,0
7,1,25.0,38600.0,4,0
8,1,36.0,49700.0,1,0
9,1,32.0,51700.0,1,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


###  Предобработка данных

#### Обработка дубликатов

In [5]:
data.duplicated().sum()

153

In [6]:
data = data.drop_duplicates().reset_index(drop=True)
data.shape

(4847, 5)

#### Обработка пропусков

In [7]:
data.isnull().sum()*100/len(data)

Пол                  0.0
Возраст              0.0
Зарплата             0.0
Члены семьи          0.0
Страховые выплаты    0.0
dtype: float64

#### Изменение типов и переименование столбцов

In [8]:
data.columns = ['пол', 'возраст', 'зарплата', 'члены_семьи', 'страховые_выплаты']

In [9]:
data['возраст'] = data['возраст'].astype('int')
data['зарплата'] = data['зарплата'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4847 entries, 0 to 4846
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   пол                4847 non-null   int64
 1   возраст            4847 non-null   int32
 2   зарплата           4847 non-null   int32
 3   члены_семьи        4847 non-null   int64
 4   страховые_выплаты  4847 non-null   int64
dtypes: int32(2), int64(3)
memory usage: 151.6 KB


### Выводы по разделу

* Удалены явные дубликаты в данных
* Названия колонок приведены к нижнему регистру
* Изменены типы колонок "возраст" и "зарплата" на целочисленный
* Пропусков в данных нет 

## Проверка изменений качества модели при умножении матрицы признаков на обратимую 

* Формула для находжения весов линейной регресси с минимальным mse:
$$w = (X^T X)^{-1} X^T y$$
* Веса при умножении на обратимую матрицу А:
$$w_1 = ((XA)^T XA)^{-1} (XA)^T y $$
$$\Leftrightarrow$$
$$w_1 = (A^T (X^T X) A)^{-1} (XA)^T y$$
$$\Leftrightarrow$$
$$w_1 = (A^T (X^T X) A)^{-1} A^T X^T y$$
$$\Leftrightarrow$$
$$w_1 = A^{-1} (X^T X)^{-1} (A^T)^{-1} A^T X^T y$$
$$\Leftrightarrow$$
$$w_1 = A^{-1} (X^T X)^{-1} E X^T y$$
$$\Leftrightarrow$$
$$w_1 = A^{-1}w$$
* Формула предсказаний для линейной регрессии:
$$a=Xw$$
* Формула для признаков умноженных на обратимую матрицу:
$$a_1=XAw_1$$
$$\Leftrightarrow$$
$$a_1=XAA^{-1}w$$
$$\Leftrightarrow$$
$$a_1=Xw=a$$
* Значит точность предсказаний не изменится.

## Алгоритм преобразования данных

* Создание функции для обучения модели и вывода результатов
* Разбить датасет на признаки и целевой признак
* Получить модель обученную на исходных данных
* Умножить матрицу признаков на обратимую случайную матрицу
* Получить модель обученную на измененных данных
* Сравнить качетво обученных моделей

Так как при умножении на обратимую матрицу качество модели не изменится, то и при следовании этому алгоритму качество модели не должно измениться.

## Проверка алгоритма

In [10]:
def get_results(features, target):
    model = LinearRegression()
    result = cross_val_score(model, features, target, cv=5)
    return result

In [11]:
features = data.drop('страховые_выплаты', axis=1)
target = data['страховые_выплаты']

In [12]:
size = features.shape[1]
matrix = np.random.rand(size, size)
matrix

array([[0.85563938, 0.04773276, 0.11382957, 0.1542869 ],
       [0.17470447, 0.21397512, 0.37507957, 0.748375  ],
       [0.88818107, 0.28095546, 0.4987011 , 0.58864296],
       [0.64165575, 0.26403354, 0.22931828, 0.42534807]])

In [13]:
np.linalg.inv(matrix)

array([[ 1.34226405, -0.17182428, -0.25375442,  0.16660704],
       [-4.85530992, -3.16925758,  0.32438634,  6.88837286],
       [-2.26224932, -1.47557534,  5.66564806, -4.42396434],
       [ 2.20870484,  3.02203962, -2.87308824,  0.20884081]])

In [14]:
result = get_results(features.values, target.values)
result

array([0.3986062 , 0.4547181 , 0.42279282, 0.41667654, 0.44617763])

In [15]:
features_new = np.dot(features, matrix)
features_new[:5]

array([[44062.44113962, 13944.47572576, 24751.2959871 , 29227.95389669],
       [33759.55863656, 10686.41449435, 18968.12479143, 22403.28313228],
       [18656.86885278,  5906.27000772, 10483.60041468, 12383.20506497],
       [37042.10263118, 11720.8643641 , 20804.17119171, 24562.97806257],
       [23187.27323327,  7338.97662817, 13026.71477642, 15384.69008013]])

In [16]:
result_new = get_results(features_new, target)
result_new

array([0.3986062 , 0.4547181 , 0.42279282, 0.41667654, 0.44617763])

### Вывод по разделу

* Алгоритм из предыдущего блока действительно работает и результаты при умножении признаков на обратимую матрицу не изменились. 

## Общий вывод

* Предобработка данных:

    * Удалены явные дубликаты в данных
    * Названия колонок приведены к нижнему регистру
    * Изменены типы колонок "возраст" и "зарплата" на целочисленный
    * Пропусков в данных нет 

* Выведено правило исходя из которой у модели линейной регрессии не измениться качество если умножить матрицу признаков на обратимую.
* Создан алгоритм для преобразования исходных данных и проверки гипотезы о неизменяемости качества модели.
* Проверена гипотеза: качество модели действительно не изменилось.